
# Barbero durmiente

Para tratar el problema se  ha implementado una clase que simula la Barberria con los procesos que se ejecutan dentro de ella.
Se proporciona el esqueleto de la implementación de los procesos para que se marquen las zonas de exclusión mutua, de espera o de sincronazación. 
En el constructor de la Barberia se inicilizan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.

In [4]:
import threading
import time
import random

class Barberia(threading.Thread):
    def __init__(self):
        #mecanismos y variables de control clientes y acceso a la sala
        ####### inicializa cada uno con el tipo y valor oportuno####
        #####puedes cambiar los nombres o añadir nuevos#######
        #controla la entrada en la sala de espera
        # Solo puedo tener 3 clientes en la sala de espera. 
        # Ojo, estar en la sala != de estar en la silla 
        self.entrarEnSala=threading.Semaphore(3)
        #controla el número de clientes en la sala
        #El numero de clientes en la sala
        self.clientesEnSala=0
        #garantiza exclusion para modificacion de variables de control de sala o acceso a secciones
        self.exclusionMutuaSala=threading.Semaphore()
        
        #mecanismos y variables de contrrol del barbero
        ####### inicializa cada uno con el tipo y valor oportuno #######
        #####puedes cambiar los nombres o añadir nuevos#######
        #controla el acesso al sillon de corte, puede poner clientes en espera y ser despertados
        self.sillon = threading.Lock()
        #controla lel estado del barbero
        self.barbero_ocupado=False
        #puede poner al barbero a dormir y ser despertado
        self.barbero_dormido=True
        

    def actividadBarbero(self):
        while True:
            ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL   ###
            ### consultar en exclusion mutua los clientes qUe hay en la sala###
            ### si no hay clientes esperando se pone a dormir hasta que le despierten ####
            self.exclusionMutuaSala.acquire() 
            if self.clientesEnSala != 0:
                
                print("El barbero da paso a un cliente\n")
                ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO ###
                self.barbero_ocupado=True
                print("El barbero corta a un cliente\n")
                time.sleep(5)
                self.sillon.notifyAll()
                ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO Y AVISAR A LOS CLIENTES DE LA SALA ###
                self.barbero_ocupado=False
                self.clientesEnSala -=1 
            else:
                self.barbero_ocupado=False
                self.barbero_dormido=True
            self.exclusionMutuaSala.release()
            
    def actividadCliente(self,name):   
        ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL   ###  
        print("El cliente "+threading.currentThread().getName()+" llega a la puerta de la barbería \n")
        # El cliente intenta entrar en la sala 
        ### consultar en exclusion mutua el estado ddel barbero###
        ### según estado entrar en la sala o no ####
        self.exclusionMutuaSala.acquire() #Para que no entre ni salga nadie
        if self.clientesEnSala < 3: #Si no hay 3 esperando, puedo entrar. 
            self.entrarEnSala.acquire()
            self.clientesEnSala +=1 
        else:
            #Aqui podria poner un bucle para que espern en la puera
            print("El cliente "+threading.currentThread().getName()+" se marcha por estar lleno \n")
        
        if self.barbero_dormido:
            self.barbero_dormido=False
        
        self.exclusionMutuaSala.release() # Los demas pueden entrar o no a la sala
           
        if self.barbero_ocupado:
            print ("Esta en el sillon el "+threading.currentThread().name+"\n")
            self.sillon.wait()
            time.sleep(3)
        else:
            
            print ("El  cliente"+threading.currentThread().name+" se corta el pelo\n")
            self.barbero_ocupado = True
            
        print ("El "+threading.currentThread().name+" abandona la barbería\n")
        
        self.entrarEnSala.release()
            
        

        

if __name__=="__main__":
        barberia= Barberia()
        cliente1 = barberia.actividadCliente('e')
        cliente1 = barberia.actividadCliente('e')
                                             
        ### INCLUYE EL CÓDIGO NECESARIO PARA CREAR E INICIALIZAR LOS HILOS  ###
       
        #barbero = 
        
        ### clientes####




El cliente MainThread llaga a la puerta de la barbería 

Esta en el sillon el MainThread

El MainThread abandona la barbería



In [2]:
test = Barberia()


In [3]:
test.entrarEnSala.acquire()


True

In [4]:
test.entrarEnSala.acquire()

True

In [5]:
test.entrarEnSala.acquire()

True

In [6]:
test.entrarEnSala.acquire()

True

In [7]:
test.entrarEnSala.acquire()

In [14]:
from threading import Thread, Lock, Event
import time, random

mutex = Lock()

#Interval in seconds
customerIntervalMin = 1
customerIntervalMax = 2
haircutDurationMin = 3
haircutDurationMax = 4

class BarberShop:
	waitingCustomers = []

	def __init__(self, barber, numberOfSeats):
		self.barber = barber
		self.numberOfSeats = numberOfSeats

	def openShop(self):
		print('Barber shop is opening')
		workingThread = Thread(target = self.barberGoToWork)
		workingThread.start()

	def barberGoToWork(self):
		while True:
			mutex.acquire()

			if len(self.waitingCustomers) > 0:
				c = self.waitingCustomers[0]
				del self.waitingCustomers[0]
				mutex.release()
				self.barber.cutHair(c)
			else:
				mutex.release()
				print ('Aaah, all done, going to sleep')
				barber.sleep()
				print ('Barber woke up')
	
	def enterBarberShop(self, customer):
		mutex.acquire()
		print ('>> {0} entered the shop and is looking for a seat'.format(customer.name))

		if len(self.waitingCustomers) == self.numberOfSeats:
			print ('Waiting room is full, {0} is leaving.'.format(customer.name))
			mutex.release()
		else:
			print ('{0} sat down in the waiting room'.format(customer.name))
			self.waitingCustomers.append(c)	
			mutex.release()
			barber.wakeUp()

class Customer:
	def __init__(self, name):
		self.name = str(name)

class Barber:
	barberWorkingEvent = Event()

	def sleep(self):
		self.barberWorkingEvent.wait()

	def wakeUp(self):
		self.barberWorkingEvent.set()

	def cutHair(self, customer):
		#Set barber as busy 
		self.barberWorkingEvent.clear()

		print ('{0} is having a haircut'.format(customer.name))

		randomHairCuttingTime = random.randrange(haircutDurationMin, haircutDurationMax+1)
		time.sleep(randomHairCuttingTime)
		print ('{0} is done'.format(customer.name))


if __name__ == '__main__':
    customers = []
    for i in range(0,5):
        customers.append(Customer(i))

        barber = Barber()

        barberShop = BarberShop(barber, numberOfSeats=3)
        barberShop.openShop()

        while len(customers) > 0:
            c = customers.pop()	
            #New customer enters the barbershop
            barberShop.enterBarberShop(c)
            customerInterval = random.randrange(customerIntervalMin,customerIntervalMax+1)
            time.sleep(customerInterval)

Barber shop is opening
Aaah, all done, going to sleep
>> 0 entered the shop and is looking for a seat
0 sat down in the waiting room
Barber woke up
0 is having a haircut
Barber shop is opening
Aaah, all done, going to sleep
>> 1 entered the shop and is looking for a seat
1 sat down in the waiting room
Barber woke up
1 is having a haircut
0 is done
Aaah, all done, going to sleep
Barber shop is opening
Aaah, all done, going to sleep
>> 2 entered the shop and is looking for a seat
2 sat down in the waiting room
Barber woke up
2 is having a haircut
Barber woke up
Aaah, all done, going to sleep
Barber shop is opening
Aaah, all done, going to sleep
>> 3 entered the shop and is looking for a seat
3 sat down in the waiting room
Barber woke up
3 is having a haircut
Barber woke up
Aaah, all done, going to sleep
1 is done
Aaah, all done, going to sleep
2 is done
Aaah, all done, going to sleep
Barber shop is opening
Aaah, all done, going to sleep
>> 4 entered the shop and is looking for a seat
4 s